In [1]:
import matplotlib.pyplot as plt
import torch
import ehc_sn as ehc

In [2]:
from ipywidgets import interact, IntSlider, FloatSlider
from functools import partial

IntSlider = partial(IntSlider, continuous_update=False)
FloatSlider = partial(FloatSlider, continuous_update=False)

In [3]:
class Model(ehc.Network):
    def step(self, x: torch.Tensor) -> torch.Tensor:
        # Excitatory layer dynamics
        xee = self.layers["excitatory"].nodes[0] @ self.w["excitatory"]["excitatory"].T
        xie = self.layers["inhibitory"].nodes[0] @ self.w["inhibitory"]["excitatory"].T
        x_e, cell = x + xee - xie, self.layers["excitatory"].cell
        self.layers["excitatory"].nodes = cell(x_e, self.layers["excitatory"].nodes[1])
        # Inhibitory layer dynamics
        xei = self.layers["excitatory"].nodes[0] @ self.w["excitatory"]["inhibitory"].T
        xii = self.layers["inhibitory"].nodes[0] @ self.w["inhibitory"]["inhibitory"].T
        x_i, cell = xei - xii, self.layers["inhibitory"].cell
        self.layers["inhibitory"].nodes = cell(x_i, self.layers["inhibitory"].nodes[1])
        # Update the synaptic weights
        self.plasticity("inhibitory", "excitatory")
        # Return the excitatory layer output
        return self.layers["excitatory"].nodes[0]

In [4]:
import tomllib as toml

with open("configurations/experiment_20250312.toml", "rb") as f:
    data = toml.load(f)

In [5]:
def plot_raster(spikes, title):
    plt.figure(figsize=(10, 5))
    for neuron_idx in range(spikes.shape[1]):
        spike_times = torch.nonzero(spikes[:, neuron_idx]).squeeze()
        plt.scatter(spike_times, neuron_idx * torch.ones_like(spike_times), s=1)
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Neuron Index")
    plt.xlim(0, spikes.shape[0])
    plt.ylim(0, spikes.shape[1])
    plt.show()

In [6]:
model = Model(p=ehc.NetworkParameters.model_validate(data))
model

Model()

In [7]:
@interact(
    current=FloatSlider(min=0, max=10, step=0.1, value=2.4),
    t1=IntSlider(min=0, max=1000, step=1, value=100),
    t2=IntSlider(min=0, max=1000, step=1, value=300),
)
def experiment(current, t1, t2):
    model.reset()
    input_current = torch.zeros(1000, model.layers["excitatory"].population).to(ehc.device)
    input_current[t1:t2, :] = current  # Apply current to the first neuron
    outputs = model(input_current)
    plot_raster(outputs.to("cpu"), "Balanced state spikes")

interactive(children=(FloatSlider(value=2.4, continuous_update=False, description='current', max=10.0), IntSli…